In [1]:
!pip install --upgrade -q git+https://github.com/keras-team/keras-cv wandb
!git clone https://github.com/soumik12345/wandb-addons -b keras/yolov8 && cd wandb-addons && pip install -q -e .

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.1/880.1 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.5/188.5 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.6/215.6 kB 17.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.4 MB/s eta 0:00:00
Cloning into 'wandb-addons'...
remote: Enumerating objects: 1842, done.
remote: Counting objects: 100% (455/455), done.
remote: Compressing objects: 100% (265/265), done.
remote: Total 1842 (delta 257), reused 259 (delta 184), pack-reused 1387
Receiving objects: 100% (1842/1842), 5.02 MiB | 23.78 MiB/s, done.
Resolving deltas: 100% (991/991), done.
  Installing build dependencies ... done
  Checking if build backend sup

In [1]:
import os

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

import keras_cv
import numpy as np

from tqdm.auto import tqdm
import matplotlib.pyplot as plt

import wandb
from wandb_addons.keras.detection import visualize_dataset

Using TensorFlow backend


In [2]:
wandb.init(
    project="keras-cv-callbacks", entity="geekyrakshit", job_type="detection"
)

config = wandb.config
config.batch_size = 4
config.base_lr = 0.005
config.model_name = "retinanet_resnet50_pascalvoc"
config.momentum = 0.9
config.global_clipnorm = 10.0


class_ids = [
    "Aeroplane",
    "Bicycle",
    "Bird",
    "Boat",
    "Bottle",
    "Bus",
    "Car",
    "Cat",
    "Chair",
    "Cow",
    "Dining Table",
    "Dog",
    "Horse",
    "Motorbike",
    "Person",
    "Potted Plant",
    "Sheep",
    "Sofa",
    "Train",
    "Tvmonitor",
    "Total",
]
config.class_mapping = dict(zip(range(len(class_ids)), class_ids))

wandb: Currently logged in as: geekyrakshit. Use `wandb login --relogin` to force relogin


In [3]:
def unpackage_raw_tfds_inputs(inputs, bounding_box_format):
    image = inputs["image"]
    boxes = keras_cv.bounding_box.convert_format(
        inputs["objects"]["bbox"],
        images=image,
        source="rel_yxyx",
        target=bounding_box_format,
    )
    bounding_boxes = {
        "classes": tf.cast(inputs["objects"]["label"], dtype=tf.float32),
        "boxes": tf.cast(boxes, dtype=tf.float32),
    }
    return {
        "images": tf.cast(image, tf.float32),
        "bounding_boxes": bounding_boxes
    }


def load_pascal_voc(split, dataset, bounding_box_format):
    ds = tfds.load(dataset, split=split, with_info=False, shuffle_files=True)
    ds = ds.map(
        lambda x: unpackage_raw_tfds_inputs(
            x, bounding_box_format=bounding_box_format
        ),
        num_parallel_calls=tf.data.AUTOTUNE,
    )
    return ds


train_ds = load_pascal_voc(
    split="train", dataset="voc/2007", bounding_box_format="xywh"
)
eval_ds = load_pascal_voc(split="test", dataset="voc/2007", bounding_box_format="xywh")

train_ds = train_ds.shuffle(config.batch_size * 4)

In [4]:
train_ds = train_ds.ragged_batch(config.batch_size, drop_remainder=True)
eval_ds = eval_ds.ragged_batch(config.batch_size, drop_remainder=True)


visualize_dataset(
    dataset=train_ds,
    class_mapping=config.class_mapping,
    title="Train-Dataset",
    max_batches_to_visualize=2
)
visualize_dataset(
    dataset=eval_ds,
    class_mapping=config.class_mapping,
    title="Eval-Dataset",
    max_batches_to_visualize=2
)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
augmenter = keras.Sequential(
    layers=[
        keras_cv.layers.RandomFlip(mode="horizontal", bounding_box_format="xywh"),
        keras_cv.layers.JitteredResize(
            target_size=(640, 640), scale_factor=(0.75, 1.3), bounding_box_format="xywh"
        ),
    ]
)

train_ds = train_ds.map(augmenter, num_parallel_calls=tf.data.AUTOTUNE)
visualize_dataset(
    dataset=train_ds,
    class_mapping=config.class_mapping,
    title="Augmented-Train-Dataset",
    max_batches_to_visualize=2
)

  0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
inference_resizing = keras_cv.layers.Resizing(
    640, 640, bounding_box_format="xywh", pad_to_aspect_ratio=True
)
eval_ds = eval_ds.map(inference_resizing, num_parallel_calls=tf.data.AUTOTUNE)
visualize_dataset(
    dataset=eval_ds,
    class_mapping=config.class_mapping,
    title="Resized-Train-Dataset",
    max_batches_to_visualize=2
)

  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
def dict_to_tuple(inputs):
    return inputs["images"], keras_cv.bounding_box.to_dense(
        inputs["bounding_boxes"], max_boxes=32
    )


train_ds = train_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)
eval_ds = eval_ds.map(dict_to_tuple, num_parallel_calls=tf.data.AUTOTUNE)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
eval_ds = eval_ds.prefetch(tf.data.AUTOTUNE)

In [8]:
pretrained_model = keras_cv.models.RetinaNet.from_preset(
    config.model_name, bounding_box_format="xywh"
)

optimizer = keras.optimizers.SGD(
    learning_rate=config.base_lr,
    momentum=config.momentum,
    global_clipnorm=config.global_clipnorm
)

coco_metrics = keras_cv.metrics.BoxCOCOMetrics(
    bounding_box_format="xywh", evaluate_freq=20
)

pretrained_model.compile(
    classification_loss="focal",
    box_loss="smoothl1",
    optimizer=optimizer,
    metrics=[coco_metrics],
)

In [9]:
coco_metrics.reset_state()
result = pretrained_model.evaluate(eval_ds.take(40))
result = coco_metrics.result(force=True)
wandb.log({f"Evaluation/{k}": v.numpy() for k, v in result.items()})

40/40 [==============================] - 365s 9s/step - loss: 0.4376 - box_loss: 0.2250 - classification_loss: 0.2126 - MaP: 0.2249 - MaP@[IoU=50]: 0.3441 - MaP@[IoU=75]: 0.2575 - MaP@[area=small]: 0.0202 - MaP@[area=medium]: 0.1558 - MaP@[area=large]: 0.2401 - Recall@[max_detections=1]: 0.1955 - Recall@[max_detections=10]: 0.2437 - Recall@[max_detections=100]: 0.2442 - Recall@[area=small]: 0.0201 - Recall@[area=medium]: 0.1612 - Recall@[area=large]: 0.2595 - percent_boxes_matched_with_anchor: 0.9018


In [15]:
wandb.finish()

Evaluation/MaP,▁
Evaluation/MaP@[IoU=50],▁
Evaluation/MaP@[IoU=75],▁
Evaluation/MaP@[area=large],▁
Evaluation/MaP@[area=medium],▁
Evaluation/MaP@[area=small],▁
Evaluation/Recall@[area=large],▁
Evaluation/Recall@[area=medium],▁
Evaluation/Recall@[area=small],▁
Evaluation/Recall@[max_detections=100],▁
Evaluation/Recall@[max_detections=10],▁
